In [1]:
# imports
from scipy.io import loadmat
import numpy as np
import pandas as pd

In [2]:
# load data
matfile_baseline = loadmat('data/raw/Ach-AT/0_0315_ach-at_0.mat')
matfile_ach = loadmat('data/raw/Ach-AT/0_0315_ach-at_1.mat')
matfile_at = loadmat('data/raw/Ach-AT/0_0315_ach-at_2.mat')

In [3]:
def make_windows(matfile, window_size = 6000):
    # retrieve the relevant data from the dictionaries
    df = pd.DataFrame(matfile['filt_data'].copy())
    df = df.T
    time = pd.DataFrame(matfile['filt_t'].copy())
    time = time.T

    # set up the time column
    time.columns = ['t']
    time['t'] = time['t'] - 60000 # make time start at 0
    time['window_id'] = time['t'] / window_size # n second windows
    time['window_id'] = time['window_id'].astype(int)

    # join the electrode signals with time
    df = pd.concat([time, df], axis=1)

    return df

In [4]:
df_base = make_windows(matfile_baseline)
df_ach = make_windows(matfile_ach)
df_at = make_windows(matfile_at)

In [5]:
# train on four electrodes - choose 4,27,34 and 57 (the centre-most ones)
df_base_4 = df_base[['t','window_id',4,27,34,57]]
df_base_4['y'] = 0
df_ach_4 = df_ach[['t','window_id',4,27,34,57]]
df_ach_4['y'] = 1
df_at_4 = df_at[['t','window_id',4,27,34,57]]
df_at_4['y'] = 2
# this could be done more efficiently, build into the function

df_4 = pd.concat([df_base_4, df_ach_4, df_at_4], axis=0, ignore_index=True)

In [6]:
# READY TO TRAIN
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters

X = extract_features(df_4, column_id='window_id', column_sort='t')

Feature Extraction:   0%|          | 0/10 [53:45<?, ?it/s]Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/site-packages/tsfresh/utilities/distribution.py", line 39, in _function_with_partly_reduce
    results = list(itertools.chain.from_iterable(results))
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/site-packages/tsfresh/utilities/distribution.py", line 38, in <genexpr>
    results = (map_function(chunk, **kwargs) for chunk in chunk_list)
  File "/home/kartikey/anaconda3/envs/ts/lib/python3.8/site-package

KeyboardInterrupt: 